# Importaciones y Funciones

In [143]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras import models, layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
from sklearn.model_selection import KFold

In [144]:
# Dicciónario para convertir los valores a categorías
LABELS = {0: 'LowQuality', 1: 'Average', 2: 'HighQuality'}
BINS = [0, 4.5, 5.5, 10]

In [145]:
def cargar_datos(dataset):
    df = pd.read_csv(dataset, sep=';')
    df['quality'] = pd.cut(df['quality'], bins=BINS, labels=[0, 1, 2])
    return df

In [146]:
def dividir_x_y(df):
    x = df.drop('quality', axis=1)
    y = df['quality']
    return x, y

In [147]:
def normalizar(x, y):
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x)
    y_onehot = keras.utils.to_categorical(y)
    return x_scaled, y_onehot

In [148]:
def clasificacion(X_scaled, y_onehot, fun_act):
    activation_fn = fun_act

    # Crear el modelo de clasificación
    classification_model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation=activation_fn, input_shape=(X_scaled.shape[1],)),
        tf.keras.layers.Dense(32, activation=activation_fn),
        tf.keras.layers.Dense(3, activation='softmax')
    ])

    # Compilar el modelo de clasificación
    classification_model.compile(optimizer='adam',
                                loss='categorical_crossentropy',
                                metrics=['accuracy'])

    # K-fold cross validation para el modelo de clasificación
    k = 5
    kf = KFold(n_splits=k, shuffle=True)

    acc = []
    loss = []

    for train_index, val_index in kf.split(X_scaled):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y_onehot[train_index], y_onehot[val_index]
        
        classification_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0) # type: ignore
        
        val_predictions = classification_model.predict(X_val)
        val_predictions = np.argmax(val_predictions, axis=1)
        val_true_labels = np.argmax(y_val, axis=1)
        
        acc.append(accuracy_score(val_true_labels, val_predictions))
        loss.append(classification_model.evaluate(X_val, y_val, verbose=0)[0]) # type: ignore
        
    print('\nPrecision con función de activación ',fun_act,': ', np.mean(acc))
    print('Costo de perdida con función de activación ',fun_act,': ', np.mean(loss))
    return np.mean(acc), np.mean(loss)

In [149]:
def regresion(X_scaled, y, fun_act):
    activation_fn = fun_act

    # Crear el modelo de regresión
    regression_model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation=activation_fn, input_shape=(X_scaled.shape[1],)),
        tf.keras.layers.Dense(32, activation=activation_fn),
        tf.keras.layers.Dense(1)
    ])

    # Compilar el modelo de regresión
    regression_model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])

    # K-fold cross validation para el modelo de regresión
    k = 5
    kf = KFold(n_splits=k, shuffle=True)

    mse_scores = []
    mae_scores = []

    for train_index, val_index in kf.split(X_scaled):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        regression_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
        
        val_predictions = regression_model.predict(X_val).flatten()
        mse = mean_squared_error(y_val, val_predictions)
        mae = mean_absolute_error(y_val, val_predictions)
        mse_scores.append(mse)
        mae_scores.append(mae)

    print('\nMSE con función de activación ',fun_act,': ', np.mean(mse_scores))
    print('MAE con función de activación ',fun_act,': ', np.mean(mae_scores))
    return np.mean(mse_scores), np.mean(mae_scores)

In [150]:
def tabla_comparative(sigmoide, tanh, val1, val2):
    return pd.DataFrame(
        {
            'Función de activación': [val1, val2],
            'Sigmoid': [sigmoide[0], sigmoide[1]],
            'Tanh': [tanh[0], tanh[1]]
        }
    )

# Vino Blanco

In [151]:
vino_blanco = cargar_datos('Data/winequality-white.csv')

In [152]:
x, y = dividir_x_y(vino_blanco)
x_scaled, y_onehot = normalizar(x, y)

## Modelo de clasificacion usando Red Neuronal

In [153]:
clas_sigmoide = clasificacion(x_scaled, y_onehot, 'sigmoid')

31/31 [==============================] - 0s 2ms/step

Precision con función de activación  sigmoid :  0.7407175168330866
Costo de perdida con función de activación  sigmoid :  0.5927406549453735


In [154]:
clas_tanh = clasificacion(x_scaled, y_onehot, 'tanh')

31/31 [==============================] - 0s 2ms/step

Precision con función de activación  tanh :  0.7615350941193637
Costo de perdida con función de activación  tanh :  0.5459101915359497


## Modelo de Regresion usando Red Neuronal

In [155]:
reg_sigmoide = regresion(x_scaled, y, 'sigmoid')

31/31 [==============================] - 0s 2ms/step

MSE con función de activación  sigmoid :  0.22416219583833213
MAE con función de activación  sigmoid :  0.3681541900716548


In [156]:
reg_tanh = regresion(x_scaled, y, 'tanh')

31/31 [==============================] - 0s 2ms/step

MSE con función de activación  tanh :  0.2155679488852804
MAE con función de activación  tanh :  0.3518212016628455


## Análisis de los Resultados

### Modelo de Classification

In [157]:
tabla_comparative(clas_sigmoide, clas_tanh, 'Precisión', 'Costo de perdida')

,Función de activación,Sigmoid,Tanh
0,Precisión,0.740718,0.761535
1,Costo de perdida,0.592741,0.545910


### Modelo de Regression

In [158]:
tabla_comparative(reg_sigmoide, reg_tanh, 'MSE', 'MAE')

,Función de activación,Sigmoid,Tanh
0,MSE,0.224162,0.215568
1,MAE,0.368154,0.351821


# Vino Tinto

In [159]:
vino_tinto = cargar_datos('Data/winequality-red.csv')

In [160]:
x, y = dividir_x_y(vino_tinto)
x_scaled, y_onehot = normalizar(x, y)

## Modelo de clasificacion usando Red Neuronal

In [161]:
clas_sigmoide = clasificacion(x_scaled, y_onehot, 'sigmoid')

10/10 [==============================] - 0s 1ms/step

Precision con función de activación  sigmoid :  0.7179408307210031
Costo de perdida con función de activación  sigmoid :  0.6413689255714417


In [162]:
clas_tanh = clasificacion(x_scaled, y_onehot, 'tanh')

10/10 [==============================] - 0s 1ms/step

Precision con función de activación  tanh :  0.7367202194357366
Costo de perdida con función de activación  tanh :  0.5920578002929687


## Modelo de Regresion usando Red Neuronal

In [163]:
reg_sigmoide = regresion(x_scaled, y, 'sigmoid')

10/10 [==============================] - 0s 1ms/step

MSE con función de activación  sigmoid :  0.2409206494804303
MAE con función de activación  sigmoid :  0.38993270991422635


In [164]:
reg_tanh = regresion(x_scaled, y, 'tanh')

10/10 [==============================] - 0s 1ms/step

MSE con función de activación  tanh :  0.22325922474196708
MAE con función de activación  tanh :  0.3649788224007234


## Análisis de los Resultados

### Modelo de Classification

In [165]:
tabla_comparative(clas_sigmoide, clas_tanh, 'Precisión', 'Costo de perdida')

,Función de activación,Sigmoid,Tanh
0,Precisión,0.717941,0.736720
1,Costo de perdida,0.641369,0.592058


### Modelo de Regression

In [166]:
tabla_comparative(reg_sigmoide, reg_tanh, 'MSE', 'MAE')

,Función de activación,Sigmoid,Tanh
0,MSE,0.240921,0.223259
1,MAE,0.389933,0.364979
